Some imports and lang features // Not all in use right now

In [9]:
:opt no-lint
{-# LANGUAGE DefaultSignatures #-}
{-# LANGUAGE GADTs #-}
{-# LANGUAGE GeneralizedNewtypeDeriving #-}
{-# LANGUAGE NamedFieldPuns#-}
import Control.Monad.State -- (StateT)
import Control.Monad.Identity
import Debug.Trace
import Data.Maybe
import Data.Bits
import Data.List
import Data.Function (fix)

Types

In [10]:
data CoderState = CS {
  ringSize :: Int, -- (modsize for bucket select)
  start :: Int, -- (how far in a stream)
  buckets :: [Int], -- (bucket starts)
  output :: String, -- (output)
  key :: String } -- (key)
  deriving (Show,Eq)

Some utility functions and assorted debris

In [11]:
memoize :: (Num a1, Enum a1) => (a1 -> a2) -> Int -> a2
memoize f = (map f [0 ..] !!)

fib :: (Eq a, Num a, Num p) => (a -> p) -> a -> p
fib f 0 = 0
fib f 1 = 1
fib f n = f (n - 1) + f (n - 2)

fibMemo :: Int -> Integer
fibMemo = fix (memoize . fib)

whichBucket :: Integral a => a -> a
whichBucket n = mod n 5

pickBucket :: [a] -> Int -> a
pickBucket buckets = (buckets !!)

modifyBucket :: [a] -> Int -> a -> [a]
modifyBucket xs n newElement = take n xs ++ [newElement] ++ drop (n + 1) xs

getInPos :: Char -> Int
getInPos l = fromMaybe 0 $ elemIndex l alphabet

-- deomstrating we could use an arbitrary indexed sequence  like n-th fib to further 'scramble' our stuff
doRing :: Integral p => Int -> p -> p
doRing _ 0 = 0
doRing index ring = (mod (fromIntegral (fibMemo index)) ring)

Some setup work

In [12]:
-- scrambled
--alphabet    = "BHu!x.VJeWaXQdiwtfpK#LTMYkbO gEs$ySDr@Io&mzRlGhF,ZcCnPv*qjNUA?"
--alphabetOut = "BHu0x1VJeWaXQdiwtfpK9LTMYkbO8gEs3ySDr4Io7mzRlGhF6ZcCnPv5qjNUA2"
-- full
--alphabet    = " !@#$,.'*?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"
--alphabetOut = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"
  
-- length 59 -- since in the liber we don't see f y z in the text on pages 49-51
alphabet    = " !fyz,.'*?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdeghijklmnopqrstuvwx"
alphabetOut = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdeghijklmnopqrstuvwx"
-- CS (modsize for bucket select) (how far in a stream) (bucket starts) (output) (key)
startCS key  = CS 11 1200 [2, 3, 5, 7, 11] "" $ cycle key
noRingCS key = CS 0    0 [0, 1, 2, 3, 4] "" $ cycle key

Ok now the worky bois

In [13]:
decoder :: [Char] -> CoderState -> [Char]
decoder (b:x:xs) cs@CS{start,buckets,key=(k:ks),output} = let
  bucketIndex = read [b] :: Int
  bucket = pickBucket buckets bucketIndex
  code = (fromMaybe 0 $ elemIndex k alphabet)
  cipher = (fromMaybe 0 $ elemIndex x alphabetOut)
  modifiedBuckets = modifyBucket buckets bucketIndex (code + bucket)
  newX = alphabet !! (mod (cipher - (code + bucket + start)) (length alphabet))
  in
    decoder xs cs{start=start+1,buckets=modifiedBuckets,key=ks,output=(newX:output)}
decoder _ cs@CS{output} = reverse output

encoder :: [Char] -> CoderState -> [Char]
encoder (x:xs) cs@CS{ringSize,start,buckets,key=(k:ks),output} = let
  code = getInPos k
  plain = getInPos x
  bucketIndex = whichBucket (getInPos k + doRing start ringSize)
  bucket = pickBucket buckets bucketIndex
  modifiedBuckets = modifyBucket buckets bucketIndex (code + bucket)
  newX = alphabetOut !! (mod (code + bucket + plain + start) (length alphabet))
  in 
    encoder xs cs{start=start+1,buckets=modifiedBuckets,key=ks,output=((reverse $ show bucketIndex ++ [newX]) ++ output)}
encoder [] cs@CS{output} = reverse output

Time for fun now

In [14]:
aKey = "ThIs ciphEr iSnt thAt haRd prObably"
aMessage = "If you can read this, then what are you doing with your life?"

Lets do it

In [15]:
encoder aMessage (startCS aKey)
-- One more time to store

cipheredText = encoder aMessage (startCS aKey)

"2D2e1i0M0A0u4L2E0M4R4W1k0E0d2B000u4H1Q4e2b1k1O1o400c3s4B4o49463Z0B0F074k4W0b1C0o09044A2R2Z0B0A1r1X3P1Z1d2l1H1P4Y0912030A0J"

In [16]:
decoder cipheredText (startCS aKey)

"If you can read this, then what are you doing with your life?"